In [1]:
import numpy as np
import pandas as pd
import torch
import csv
import transformers
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from torch.optim import AdamW
import json
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification,AutoConfig, get_linear_schedule_with_warmup


In [2]:
device = 'cuda'

In [3]:
#MODEL_NAME = "roberta-base"
MODEL_NAME = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
MAX_LEN = 64
EPOCHS = 10
BATCH_SIZE = 32 
LR = 2e-5 
# WARMUP_STEPS = 100 
# T_TOTAL = 1000 

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [5]:
def load_dataset(filepath, max_len):
    label = []
    sentences = []
    # load dataset
    f = open(filepath, 'r', encoding='utf-8')
    r = csv.reader(f)
    for item in r:
        if r.line_num == 1:
            continue
        #print(item)
    
        label.append(int(item[2]))
        sentences.append(item[1])
        
    input_ids = []
    attention_masks = []

    # For every sentence...
    for data in sentences:
        encoded_data = tokenizer.encode_plus(
            text=data,                      # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=max_len,             # Max length to truncate/pad
            padding='max_length',           # Pad sentence to max length
            return_attention_mask=True,      # Return attention mask
            truncation= True
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_data.get('input_ids'))
        attention_masks.append(encoded_data.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    labels = torch.tensor(label)
    return input_ids, attention_masks, labels


In [6]:
import os
os.getcwd()

'f:\\Desktop\\SocialNetworkProj\\model'

In [7]:
train_dataset = load_dataset('train/train.csv', max_len = MAX_LEN)
valid_dataset = load_dataset('train/val.csv', max_len = MAX_LEN)
test_dataset = load_dataset('train/test.csv', max_len = MAX_LEN)

In [8]:
train_dataset[2]

tensor([0, 2, 0,  ..., 0, 2, 0])

In [9]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_data = TensorDataset(train_dataset[0], train_dataset[1],train_dataset[2])
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, sampler=train_sampler, batch_size = BATCH_SIZE)

val_data = TensorDataset(valid_dataset[0],valid_dataset[1],valid_dataset[2])
val_sampler = SequentialSampler(val_data)
val_loader = DataLoader(val_data,sampler=val_sampler, batch_size = BATCH_SIZE)

test_data = TensorDataset(test_dataset[0],test_dataset[1],test_dataset[2])

In [10]:
def batch_accuracy(pre, label):
    pre = pre.argmax(dim=1)
    correct = torch.eq(pre, label).sum().float().item()
    accuracy = correct / float(len(label))

    return accuracy

In [11]:
# config = AutoConfig.from_pretrained(MODEL_NAME)
# config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels = 3)
model.cuda()
print()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
optimizer = AdamW(model.parameters(), lr = LR)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)

In [13]:
from tqdm import tqdm

In [14]:
model.train()
print('training...')
best_acc = 0
for epoch in range(EPOCHS):
    loss_t, batch_loss, batch_acc, batch_counts = 0, 0, 0, 0
    
    model.train()
    for step,batch in tqdm(enumerate(train_loader), desc = "Training"):
        batch_counts +=1
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
        outputs = model(b_input_ids, b_attn_mask, labels=b_labels)
        loss, logits = outputs[:2]
        optimizer.zero_grad()
        
        loss_t += loss.item()
        batch_loss += loss.item()
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        
        acc = batch_accuracy(logits, b_labels)
        batch_acc += acc
        
        if (step % 200 == 0 and step != 0) or (step == len(train_loader) - 1):
            print(f'epoch:{epoch} | step:{step} | avg_batch_acc:{batch_acc/batch_counts:^.6f} | avg_batch_loss:{batch_loss/batch_counts:^.6f}')
            batch_acc, batch_loss, batch_counts = 0, 0, 0
        
    avg_train_loss = loss_t / len(train_loader)
    
    #evaluate 
    val_acc, val_loss = [],[]
    model.eval()
    for batch in val_loader:
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            outputs = model(b_input_ids, b_attn_mask, labels=b_labels)
        loss, logits = outputs[:2]
        val_loss.append(loss.item())
        acc = batch_accuracy(logits, b_labels)
        val_acc.append(acc)
        
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_acc)
            
    print(f'epoch:{epoch} | avg_train_loss:{avg_train_loss} | val_loss:{val_loss} | val_accuracy:{val_accuracy}')
    
    if val_accuracy>=best_acc:
        torch.save(model.state_dict(), 'bert_cla2.ckpt')
        best_acc = val_accuracy
        print('saving trained model...')


training...


Training: 201it [01:20,  2.57it/s]

epoch:0 | step:200 | avg_batch_acc:0.765858 | avg_batch_loss:0.560319


Training: 401it [02:38,  2.55it/s]

epoch:0 | step:400 | avg_batch_acc:0.767344 | avg_batch_loss:0.556397


...

Training: 601it [03:57,  2.52it/s]

epoch:9 | step:600 | avg_batch_acc:0.988281 | avg_batch_loss:0.042901


Training: 687it [04:31,  2.53it/s]

epoch:9 | step:686 | avg_batch_acc:0.982195 | avg_batch_loss:0.063108


epoch:9 | avg_train_loss:0.048667650760556104 | val_loss:1.3368132090152696 | val_accuracy:0.7950581395348837


In [15]:
print('开始加载训练完成的model...')
model.load_state_dict(torch.load('bert_cla2.ckpt'))

开始加载训练完成的model...


<All keys matched successfully>

In [16]:
print('开始测试...')
model.eval()
test_result = []
for data in test_data:
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in data)
    b_input = b_input_ids.unsqueeze(0)
    
    with torch.no_grad():
        outputs = model(b_input)
        pre = outputs.logits.argmax(dim=1)
        test_result.append([b_labels.item(), pre.item(), tokenizer.convert_ids_to_tokens(b_input_ids)])

# 写入csv文件
df = pd.DataFrame(test_result)
df.to_csv('test_result.csv',index=False, header=['id', 'label','text'])

开始测试...


In [17]:
import pandas as pd
df = pd.read_csv('test_result.csv')
df_e = df[df.id!=df.label]

In [18]:
df

,id,label,text
0,1,1,"['<s>', 'the', 'Ġgr', 'ates', 'Ġchildren', 'Ġc..."
1,0,0,"['<s>', 'going', 'Ġto', 'Ġsleep', 'Ġgonna', 'Ġ..."
2,1,1,"['<s>', 'loo', 'ool', 'Ġme', 'Ġana', 'Ġi', 'Ġw..."
3,2,1,"['<s>', 'oh', 'Ġno', 'Ġi', 'Ġhope', 'Ġyou', 'Ġ..."
4,1,1,"['<s>', 'see', 'Ġyou', 'Ġon', 'Ġmay', '</s>', ..."
...,...,...,...
2743,1,1,"['<s>', 'its', 'Ġbout', 'Ġsmoking', 'Ġweed', '..."
2744,2,2,"['<s>', 'i', 'Ġthink', 'Ġthats', 'Ġpretty', 'Ġ..."
2745,0,0,"['<s>', 'went', 'Ġto', 'Ġsleep', 'Ġand', 'Ġthe..."
2746,1,2,"['<s>', 'wh', 'ar', 'Ġa', 'Ġnight', 'Ġwoo', 'Ġ..."


In [19]:
df_e

,id,label,text
3,2,1,"['<s>', 'oh', 'Ġno', 'Ġi', 'Ġhope', 'Ġyou', 'Ġ..."
10,1,0,"['<s>', 'i', 'Ġhad', 'Ġit', 'Ġon', 'Ġmy', 'Ġit..."
14,0,2,"['<s>', 'i', 'Ġhope', 'Ġso', 'Ġrecorded', 'Ġth..."
18,1,0,"['<s>', 'i', 'Ġfeel', 'Ġsome', 'Ġtype', 'Ġof',..."
26,1,0,"['<s>', 'day', 'Ġof', 'Ġwork', 'Ġtoday', 'Ġwas..."
...,...,...,...
2722,2,1,"['<s>', 'my', 'Ġhub', 'by', 'Ġand', 'Ġhis', 'Ġ..."
2731,1,0,"['<s>', 'i', 'Ġcant', 'Ġfind', 'Ġmy', 'Ġtennis..."
2735,1,2,"['<s>', 'thanks', 'Ġf', 'ot', 'ore', 'port', '..."
2736,1,2,"['<s>', 'y', 'ep', 'Ġthat', 'Ġone', 'Ġworks', ..."


In [20]:
1- len(df_e)/len(df)

0.7914847161572053